<a href="https://colab.research.google.com/github/SANDESHAHAHA/Data-science2/blob/main/Fakenews_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News classifier using LSTM

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

In [ ]:
fake_df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "clmentbisaillon/fake-and-real-news-dataset",
    path="Fake.csv"
)

true_df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "clmentbisaillon/fake-and-real-news-dataset",
    path="True.csv"
)

/tmp/ipython-input-2324823427.py:1: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  fake_df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'fake-and-real-news-dataset' dataset.


/tmp/ipython-input-2324823427.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  true_df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'fake-and-real-news-dataset' dataset.


In [ ]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
true_df['label'] = 1
fake_df['label'] = 0

In [ ]:
fake_df.shape

(23481, 5)

In [ ]:
true_df.shape

(21417, 5)

In [ ]:
df=pd.concat([fake_df,true_df],axis=0,ignore_index=True)

In [ ]:
df.sample(5)

,title,text,subject,date,label
20760,WOW! NAVY SEAL BLASTS HILLARY: “You Are An Ign...,This is a MUST WATCH!According to American New...,left-news,"Apr 9, 2016",0
20744,THE CHALKENING: Political Chalk Drawing Bandit...,The social justice warriors are being terroriz...,left-news,"Apr 13, 2016",0
34832,Syria says military jet downed in northern Ham...,AMMAN (Reuters) - Syria s armed forces said in...,worldnews,"December 26, 2017",1
42502,Russian foreign ministry to meet visiting Nort...,MOSCOW (Reuters) - Russia s foreign ministry p...,worldnews,"September 26, 2017",1
3282,Is Trump Purging The Government? Team Asking ...,Fears of purges of personnel based on ideology...,News,"December 24, 2016",0


In [ ]:
df=df.sample(frac=1,random_state=42).reset_index(drop=True)

In [ ]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [ ]:
df.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [ ]:
new_df=df['title']

In [ ]:
new_df.head()

,title
0,Ben Stein Calls Out 9th Circuit Court: Committ...
1,Trump drops Steve Bannon from National Securit...
2,Puerto Rico expects U.S. to lift Jones Act shi...
3,OOPS: Trump Just Accidentally Confirmed He Le...
4,Donald Trump heads for Scotland to reopen a go...


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=5000

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer # for stemming running->run
ps = PorterStemmer()
corpus = []

for i in range(0,len(new_df)):
  review=re.sub('[^a-zA-Z]',' ',new_df[i])
  review=review.lower()
  review=review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['ben stein call th circuit court commit coup tat constitut',
 'trump drop steve bannon nation secur council',
 'puerto rico expect u lift jone act ship restrict',
 'oop trump accident confirm leak isra intellig russia video',
 'donald trump head scotland reopen golf resort',
 'paul ryan respond dem sit gun control disgust way video',
 'awesom diamond silk rip press believ video',
 'stand cheer ukip parti leader slam germani franc eu invas phoni refuge video',
 'north korea show sign seriou talk u offici',
 'trump signal willing rais u minimum wage',
 'new jersey christi mull run lead republican parti report',
 'hillari clinton spot dine alon',
 'franc germani want iran revers ballist missil program',
 'aid eu commiss head tweet pictur white smoke brexit meet may',
 'trump issu warn man armi could isi video',
 'u give lao extra million help clear unexplod ordnanc',
 'judg declar babi name illeg prevent emot harm',
 'paul ryan take monument humili photo constitu expertli troll imag',
 '

In [ ]:
one_hot_repr=[one_hot(words,voc_size) for words in corpus]
one_hot_repr

[[4929, 540, 4856, 517, 3549, 2145, 1021, 4627, 1925, 3799],
 [2169, 4319, 4619, 1353, 4141, 35, 314],
 [901, 3035, 1710, 3910, 3566, 1451, 2725, 2260, 228],
 [4703, 2169, 4232, 2681, 3223, 4353, 364, 1591, 1911],
 [3126, 2169, 1967, 1193, 1311, 2327, 1004],
 [1737, 4112, 4139, 1672, 4515, 4902, 2139, 1621, 4257, 1911],
 [3833, 3040, 1584, 2118, 3474, 1259, 1911],
 [2725,
  2822,
  1985,
  2641,
  1674,
  4167,
  3067,
  3447,
  3623,
  1822,
  2360,
  2224,
  1911],
 [2224, 321, 3418, 882, 3055, 4437, 3910, 3253],
 [2169, 3231, 2580, 2250, 3910, 630, 4633],
 [2622, 3691, 1707, 4080, 572, 2136, 1137, 2641, 1791],
 [4189, 1764, 525, 4035, 770],
 [3447, 3067, 2033, 484, 535, 4171, 981, 1308],
 [3177, 3623, 488, 1967, 3826, 386, 2805, 3141, 530, 2852, 3755],
 [2169, 3206, 1759, 1680, 1622, 1729, 1102, 1911],
 [3910, 1848, 4545, 1054, 3559, 3797, 1806, 1571, 2881],
 [4251, 2409, 4349, 1717, 2392, 4728, 4267, 1739],
 [1737, 4112, 2020, 1066, 3013, 2983, 2020, 995, 2253, 4499],
 [1137, 4035,

Embedding Representation

In [ ]:
sent_lengths=20
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_lengths)
print(embedded_docs)

[[   0    0    0 ... 4627 1925 3799]
 [   0    0    0 ... 4141   35  314]
 [   0    0    0 ... 2725 2260  228]
 ...
 [   0    0    0 ... 4437 1183 2575]
 [   0    0    0 ... 1811 4415 3008]
 [   0    0    0 ... 3848 2520 3437]]


In [ ]:
#creating  model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_lengths))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
len(embedded_docs)

44898

In [ ]:
y=df['label']
y.shape

(44898,)

In [ ]:
import numpy as np
X=np.array(embedded_docs)
y=np.array(y)

In [ ]:
X.shape,y.shape

((44898, 20), (44898,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8399 - loss: 0.3458 - val_accuracy: 0.9391 - val_loss: 0.1502
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9503 - loss: 0.1263 - val_accuracy: 0.9402 - val_loss: 0.1460
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9636 - loss: 0.0913 - val_accuracy: 0.9428 - val_loss: 0.1503
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9715 - loss: 0.0796 - val_accuracy: 0.9442 - val_loss: 0.1531
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9804 - loss: 0.0539 - val_accuracy: 0.9440 - val_loss: 0.1825
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9856 - loss: 0.0386 - val_accuracy: 0.9412 - val_loss: 0.2080
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9892 - loss: 0.0303 - val_accuracy: 0.9410 - val_loss: 0.2138
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9909 - loss: 0.0258 - val_accuracy: 0.

In [ ]:
y_pred=model.predict(X_test)

464/464 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[7308,  446],
       [ 458, 6605]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9389889991226295

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      7754
           1       0.94      0.94      0.94      7063

    accuracy                           0.94     14817
   macro avg       0.94      0.94      0.94     14817
weighted avg       0.94      0.94      0.94     14817

